In [83]:
#import libs
import pyodbc 
import pandas as pd
import pandas.io.sql as psql
from sqlalchemy.engine import URL
import numpy as np
import psycopg2.extras as extras

In [84]:
#extract data from source
connection_string = "DRIVER={SQL Server};SERVER=DESKTOP-MA71DLJ;DATABASE=PROJECT_PIPELINE;Trusted_Connection=yes;"
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
from sqlalchemy import create_engine
engine = create_engine(connection_url)
sql= 'SELECT * FROM dbo.sales'
df= psql.read_sql(sql,engine)


In [85]:
#add Surrogate Key
df.insert(0, 'TransactionId', range(1,1+len(df)))
df['Date']=df['Date'].str.replace('00:00:00','')

In [86]:
#Transform and split dataframe
Geo =  df.groupby(['Area_Code','State','Market','Market_Size']).size().reset_index().rename(columns={0: 'Count'})
Product = df.groupby(['ProductId','Product_Type','Product','Type']).size().reset_index().rename(columns={0: 'Count'})
for column_name in Product.columns:
    if column_name != 'Count' and column_name != 'ProductId':
        df.drop([column_name], axis =1,inplace= True)
for column_name in Geo.columns:
    if column_name != 'Count' and column_name != 'Area_Code':
        df.drop([column_name], axis =1,inplace= True)
Transactions= df.copy()
Transactions['Date'] = Transactions['Date'].apply(lambda x: x.strip('.0000000'))


In [87]:
#Load into Postgres
import psycopg2
conn= psycopg2.connect("host= localhost dbname = PROJECT_PIPELINE user = etl password = demopass" )
cur = conn.cursor()
cur.execute("""
    CREATE TABLE Dim_Product(
    ProductId integer,
    Product_Type text,
    Product text,
    Type text,
    Count integer,
    PRIMARY KEY (ProductId)
)
""")

In [88]:
cur.execute("""
    CREATE TABLE Dim_Geo(
    Area_Code integer,
    State text,
    Market text,
    Market_Size text,
    Count integer,
    PRIMARY KEY(Area_Code)
)
""")

In [89]:
cur.execute("""
   CREATE TABLE Transactions(
    TransactionId integer,
    Area_Code integer,
    ProductId integer,
    Profit decimal,
    Margin decimal,
    Sales decimal,
    COGS  decimal,
    Total_Expenses decimal,
    Marketing decimal,
    Inventory decimal,
    Budget_Profit decimal,
    Budget_COGS decimal,
    Budget_Margin decimal,
    Budget_Sales decimal,
    Date TIMESTAMP,
    PRIMARY KEY(TransactionId),
    FOREIGN KEY(Area_Code) REFERENCES Dim_Geo (Area_Code),
    FOREIGN KEY(ProductId) REFERENCES Dim_Product (ProductId)
)
""")
conn.commit()

In [90]:
import numpy as np
import psycopg2.extras as extras
def execute_values(conn, df, table):
  
    tuples = [tuple(x) for x in df.to_numpy()]
  
    cols = ','.join(list(df.columns))
    # SQL query to execute
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("the dataframe is inserted")
    cursor.close()
  
  
conn= psycopg2.connect("host= localhost dbname = PROJECT_PIPELINE user = etl password = demopass" )


In [91]:
execute_values(conn, Geo, 'Dim_Geo')
execute_values(conn, Product, 'Dim_Product')
execute_values(conn, Transactions, 'Transactions')



the dataframe is inserted
the dataframe is inserted
the dataframe is inserted
